In [1]:
#Import relevant packages
import os, sys
import arcpy
from arcpy import env
from arcpy.sa import Con, IsNull, Raster, Plus, Minus, FocalStatistics, NbrRectangle

In [6]:
#USER INPUT

#Directory for input/output files 
directory = 'D:/data/dem_kap'

#Names of DEM files to process, as list
demFiles = ['12_38_1_2_2m_v3.0_reg_dem.tif', '12_38_2_1_2m_v3.0_reg_dem.tif', 
            '12_38_2_2_2m_v3.0_reg_dem.tif']

#UAV type (wingtra/ebee)
uav = 'wingtra'

#EPSG projection 
proj = 32622

#Output cellsize
outCell=30

#Output DEM filename
outFile = 'outDEM.tif'

#Filepath to mask (e.g. land polygon)
maskFile = 'G100_landpolygon_AOI_KAP.shp'

#Invert mask file? (invert if polygon represents land, don't invert if polygon represents sea)
invert = True

#Bounding box file
boundFile = '0605KAP_BoundingBox.shp'

#Geoid file
geoidFile = 'G:/geoide_GGEIOD16/ggeoid16.tif'

#Remove intermediary files after processing?
clean=True

In [7]:
#Set ArcPy environment
env.workspace = directory
env.overwriteOutput = True
env.cellSize=outCell
env.resamplingMethod('MAXIMUM')

In [8]:
#Check DEM datums and change if incompatible
count = 1
newFiles=[]

#Loop through DEMs
for d in demFiles:
    
    #Retrieve projection
    coord = arcpy.Describe(d).spatialReference

    #Check if projection is compatible with uav type
    if uav in ['wingtra','Wingtra']:
        if coord.name in ['Polar_Stereographic']:            
            out = Minus(Raster(d), Raster(geoidFile))
            rasFile = d.split('.')[0] + '_msl.' + d.split('.')[-1]
            newFiles.append(rasFile)
            out.save(rasFile) 
        else:
            newFiles.append(d)
    else:
        if coord.name in ['Polar_Stereographic']: 
            newFiles.append(d)
        else:
            out = Plus(Raster(d), Raster(geoidFile))
            rasFile = d.split('.')[0] + '_ellip.' + d.split('.')[-1]
            newFiles.append(rasFile)
            out.save(rasFile) 
    count=count+1

In [9]:
#Merge DEMs
arcpy.MosaicToNewRaster_management(newFiles, directory,'merged_dem.tif', "", 
                                   '32_BIT_FLOAT', '', '1', 'FIRST', 'FIRST')
dem = Raster('merged_dem.tif')

In [10]:
#Mask merged DEM if mask file is provided
if maskFile is not None:
    
    #Convert vector to raster
    field = arcpy.ListFields(maskFile)[0]
    field = str(field.name)
    arcpy.PolygonToRaster_conversion(maskFile, field, 'mask_raster.tif', '', '', '')
    maskRas = Raster('mask_raster.tif')    
    
    #Mask DEM, invert if flag is True
    if invert is False:
        masked = Con(IsNull(maskRas), dem, 0)       
    else:
        masked = Con(IsNull(maskRas), 0, dem) 
        
    #Mosaic masked DEM and merged DEM (take this step out if you want the mask to dictate the DEM extent)
    arcpy.MosaicToNewRaster_management([masked,dem], directory, 'masked_dem.tif',
                                        "", '32_BIT_FLOAT', '10', '1', 'FIRST', 'FIRST')    
    dem = Raster('masked_dem.tif')

In [11]:
#Fill holes in DEM (if hole is less than 10 x 10px)
dem = Con(IsNull(dem), FocalStatistics(dem, NbrRectangle(10, 10),'MEAN'), dem)

In [12]:
#Reproject DEM if EPSG number is provided
if proj is not None:

    #Create spatial reference from EPSG
    outproj = arcpy.SpatialReference(int(proj))
    
    #Project DEM
    arcpy.ProjectRaster_management(dem, 'reprojected_dem.tif', outproj,
                                   'BILINEAR', str(outCell))
    dem = Raster('reprojected_dem.tif')

In [13]:
#Clip DEM to bounding box if provided
if boundFile is not None:
    arcpy.Clip_management(dem, '', 'clipped_dem.tif', boundFile, '', 'ClippingGeometry') 
    dem = Raster('clipped_dem.tif')

In [14]:
#Save final DEM to file
dem.save(outFile)

In [15]:
#Clean intermediary files from output folder

#Define function for removing files
def removeFile(filename):
    if os.path.isfile(filename) is True:
        try:
            os.remove(str(filename))
            print(filename + ' removed')
        except:
            print(filename + ' could not be removed. Check permissions.')
            pass

#Clean folder if flag is True
if clean is True:
    print('\nCleaning folder ' + str(directory))
    removeFile(directory + 'merged_dem.tif')
    removeFile(directory + 'mask_raster.tif')
    removeFile(directory + 'masked_dem.tif')    
    removeFile(directory + 'reprojected_dem.tif')            
    removeFile(directory + 'clipped_dem.tif')


Cleaning folder D:/data/dem_kap
